In [16]:
import pandas as pd
import numpy as np
from datetime import date

In [21]:
# 초기 tidy
def tidy(df):
    df.columns = ['방송', '식당', '방영일', '포스팅일', '제목', '주소']
    df = df[df['포스팅일'].isnull() == False]
    df['방영일'] = df['방영일'].astype(str)
    
    return df

In [17]:
# 방영일 date 형식으로 바꾸기
def get_date_from_b_date(df_column):
    b_date = []
    for data in df_column:
        year = data[:4]
        month = data[4:6]
        day = data[6:8]
        single_date = date(int(year), int(month), int(day))
        b_date.append(single_date)

    return pd.Series(b_date)

In [18]:
# 포스팅일 date 형식으로 바꾸기
def get_date_from_p_date(df_column):
    p_date = []
    for data in df_column:
        date_components = data.split('.')
        year = date_components[0]
        month = date_components[1]
        day = date_components[2]
        single_date = date(int(year), int(month), int(day))
        p_date.append(single_date)
    return pd.Series(p_date)

In [19]:
# 방영일과 포스팅일 날짜 차이 구하기
def get_days_diff(col_days_diff, col_b_day, col_p_day):
    col_days_diff = col_p_day - col_b_day
    col_days_diff = col_days_diff.astype(str).str.findall(r'(\-?\d+)\s')
    days_list = []
    for days in col_days_diff:
        days_list.append(int(days[0]))
    return days_list

In [20]:
# 위치정보 넣기
dining_areas = {
'대풍만두' :'안동' ,'큰바다해물아구찜' :'일산/파주' ,'여인천하조개구이' :'인천/을왕리/영종도' ,'제일정육정식당' :'용문/오산' ,'칡산에' :'원주' ,'동남집' :'구디/가산' ,
'찜혜화동' :'대학로/혜화' ,'통큰손칼국수' :'인천/소랲포구' ,'송씨 뼈대 있는 짬뽕' :'강원도/원주/횡성' ,'행복한 콩박사' :'오산/양산/병점' ,'아이포차' :'부천/역곡' ,'김관석의 탐라돈' :'신촌/홍대' ,
'청송숯불갈비' :'대구/대명' ,'세친구 족발이야기' :'인천/계양/작전' ,'요망진 도세기' :'제주/애월/당산' ,'돈까스잔치' :'동빙고/용산/동부' ,'송영선할머니갈비집' :'포천' ,'136생고기' :'인천/작전동/동부' ,
'영월애곤드레' :'인천/부천' ,'빽갈비1983' :'논현/강남구청' ,'마포참치' :'마포' ,'옛골매운갈비김치찜' :'성남/상대원/모란' ,'토가' :'강화도' ,'짜글이식당' :'충주/청주' ,
'번개맞은흑돼지' :'제주/서귀포' ,'섭지향' :'제주/성산/섭지코지' ,'꼬치파이터' :'전주/객사' ,'용두동쭈꾸미' :'동대문' ,'소담골' :'동두천/강화도' ,'광명소고기순대국' :'광명/철산' ,
'신촌해물칼국수' :'신촌/홍대' ,'더행복' :'상암' ,'편가네 간장게장' :'강화도' ,'행복한 곰탕' :'대전/유성/세종' ,'밴댕이소갈딱지삼겹살강가네' :'진해/창원' ,'천마루' :'무주' ,
'군자네' :'대흥/마포' ,'현래장' :'마포' ,'시내식당' :'광양/여수' ,'금목서회관' :'광양' ,'평창면옥' :'평창' ,'더코너키친' :'옥수/평창' ,
'구로식당 본점' :'구디/구로' ,'영빈관' :'원주' ,'맛자랑' :'경주' ,'할머니 묵집' :'파주' ,'구옹진식당' :'서산/충남' ,'강화해신탕' :'강화' ,
'속초어시장' :'일산' ,'약산궁 능이버섯백숙' :'포천/경기' ,'속초해물보쌈전골' :'속초' ,'제주미돈' :'인천/소래' ,'오시오집' :'영등포/여의도' ,'계림식당' :'종로' ,
'옛골할머니감자탕순대국' :'일산' ,'청담추어정' :'성남/시흥' ,'까치기사식당' :'인천/십정동' ,'시골본가집' :'남양주' ,'갈비구락부' :'울산/언양' ,'반저' :'대학로/혜화' ,
'소문난 곱창' :'마천/송파' ,'당구대통철판삼겹살' :'부산/기장' ,'전주집' :'종로/을지로' ,'원조추어탕' :'강남/역삼/신논현' ,'샤리' :'부산/서면' ,'이태리국시' :'대전/둔산' ,
'쑥고개부대찌개' :'안성/칠곡' ,'꽃삼' :'아산/신용화' ,'네모오징어 도곡본점' :'매봉/도곡' ,'연호정칼국수' :'김포/강화' ,'진심인게야' :'일산/풍동' ,'스앤샤 홍대본점' :'홍대' ,
'취향왕짬뽕' :'안양/시흥' ,'쇠뫼기' :'경기/광주/퇴촌' ,'바다담은 해물 손칼국수' :'원당/일산/고양' ,'강릉아구찜' :'신촌/이대' ,'미스테이크룸' :'이태원' ,'고기한점 달빛한점' :'하단/건대/광진구' ,
'하선재 간장게장' :'송파/방이/잠실' ,'이태원 우육미엔' :'이태원/한남' ,'푸른초장' :'충정' ,'어부네 횟집' :'마포' ,'철판패밀리' :'의정부' ,'해신247' :'아주/거제' ,
}

In [27]:
sangsang = pd.read_csv('sangsang_2017_90days.csv')

def final_tidy(df):
    df = tidy(df)
    df['방영일'] = get_date_from_b_date(df['방영일'])
    df['포스팅일'] = get_date_from_p_date(df['포스팅일'])
    df['days_diff'] = pd.Series()
    df['days_diff'] = get_days_diff(df['days_diff'], df['방영일'], df['포스팅일'])
    df['지역'] = df['식당'].map(dining_areas).astype(str)
    a = df.loc[df.apply(lambda r: ((any(area in r[4] for area in r[7].split('/'))) | (r[0] in r[4])|(r[1] in r[4])), axis=1)]
    return a

sangsang_area = final_tidy(sangsang)
sangsang_area.head(3)
# # 칼럼명, 널값 없애기
# sangsang = tidy(sangsang)

# # 방영일, 포스팅일 date 객체로 바꾸기
# sangsang['방영일'] = get_date_from_b_date(sangsang['방영일'])
# sangsang['포스팅일'] = get_date_from_p_date(sangsang['포스팅일'])

# # days_diff 구하기
# sangsang['days_diff'] = pd.Series()
# sangsang['days_diff'] = get_days_diff(sangsang['days_diff'], sangsang['방영일'], sangsang['포스팅일'])

# # 지역 맵핑하기
# sangsang['지역'] = sangsang['식당'].map(dining_areas).astype(str)

# # 식당이름, 방송이름, 지역이 포함된 row만 살리기
# sangsang_area = sangsang.loc[sangsang.apply(lambda r: ((any(area in r[4] for area in r[7].split('/'))) | (r[0] in r[4])|(r[1] in r[4])), axis=1)]

,방송,식당,방영일,포스팅일,제목,주소,days_diff,지역
0,생생정보통,대풍만두,2017-01-03,2017-01-08,안동 사대부찜닭 생생정보통에 나온 <mark>대풍만두</mark> 다녀왔어요,"https://m.blog.naver.com/hts1222/220905835536""",5,안동
1,생생정보통,대풍만두,2017-01-03,2017-01-27,"안동구시장찐빵맛집, 생생정보통 <mark>대풍만두</mark>","https://m.blog.naver.com/pinkwona/220920892215""",24,안동
2,생생정보통,대풍만두,2017-01-03,2016-12-20,"안동시 서부동 만두맛집, <mark>대풍만두</mark>","http://siksamenu.tistory.com/2212""",-14,안동
